In [1]:
# Carrega Bibliotecas

library(dplyr)
library(ggplot2)
library(daltoolbox)
library(RColorBrewer)
library(GGally)
library(reshape)
library(corrplot)
library(WVPlots)
library(aplpack)
library(gridExtra)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘daltoolbox’


The following object is masked from ‘package:base’:

    transform


Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2


Attaching package: ‘reshape’


The following object is masked from ‘package:dplyr’:

    rename


corrplot 0.92 loaded

Loading required package: wrapr


Attaching package: ‘wrapr’


The following object is masked from ‘package:dplyr’:

    coalesce


Warning message:
“no DISPLAY variable so Tk is not available”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘tidyr’


The following objects are masked from ‘package:wrapr’:

    pack, un

In [2]:
# Carrega Dataset
load("/home/data/malaria/malaria.RData")

# Lista Objetos
ls()

[1] "data"

In [3]:
# Exibe as Colunas
head(data)

,infection.county,home.county,notification.county,qty.parasites,scheme,notification.hr,home.hr,infection.hr,exam.type,exam.result,⋯,exam.month,exam.year,treatment.month,treatment.year,symptom.month,symptom.year,exam.interval,treatment.interval,notification.interval,previous.treatment
,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<ord>,<ord>,<ord>,<ord>,<ord>,<ord>,<ord>,<ord>,<ord>,<fct>
1,110001,110001,110001,501100,1,ZONA DA MATA,ZONA DA MATA,ZONA DA MATA,thick and thin blood smears,vivax,⋯,02,2014,02,2014,02,2014,on the same day,on the same day,on the same day,vivax
2,110001,110001,110001,NA,1,ZONA DA MATA,ZONA DA MATA,ZONA DA MATA,thick and thin blood smears,vivax,⋯,12,2014,12,2014,12,2014,on the same day,on the same day,1 to 7 days,vivax
3,110001,110001,110001,NA,1,ZONA DA MATA,ZONA DA MATA,ZONA DA MATA,thick and thin blood smears,vivax,⋯,11,2014,11,2014,11,2014,on the same day,on the same day,on the same day,no
4,110001,110001,110001,NA,1,ZONA DA MATA,ZONA DA MATA,ZONA DA MATA,thick and thin blood smears,vivax,⋯,10,2013,10,2013,10,2013,on the same day,on the same day,1 to 7 days,no
5,110001,110001,110001,3,1,ZONA DA MATA,ZONA DA MATA,ZONA DA MATA,thick and thin blood smears,vivax,⋯,10,2014,10,2014,10,2014,on the same day,on the same day,1 to 7 days,no
6,110001,110001,110001,305500,1,ZONA DA MATA,ZONA DA MATA,ZONA DA MATA,thick and thin blood smears,vivax,⋯,11,2013,11,2013,NA,NA,1 to 7 days,on the same day,NA,no


In [4]:
#Separa tabelas para usarmos no testarmos os algoritmos
split <- data %>% select (gender,age,exam.result)

#Aplicado filtro aos valores mais usados para o gráfico pois os demais campos eram muito irrelevantes para aparecer no gráfico
#também retira os resultados negativos

split <- split %>% filter (exam.result=="vivax" | exam.result=="falciparum")



In [5]:
summary(split)

     gender                        age             exam.result     
 male   :1604514   05 to 14 years old:621329   vivax     :2365980  
 female :1023725   15 to 24 years old:561862   falciparum: 262440  
 ignored:    175   25 to 34 years old:458624   negative  :      0  
 NA's   :      6   35 to 44 years old:320548   F+FG      :      0  
                   01 to 04 years old:268809   F+V       :      0  
                   (Other)           :397214   V+FG      :      0  
                   NA's              :    34   (Other)   :      0  

In [6]:
# Observamos que o algoritmos só aceita dados numéricos
# Transformando dados categóricos em dados numéricos

split$age <- unclass(split$age)
split$gender <- unclass(split$gender)
#tirando NA
split <- na.omit(split)
summary(split)

     gender          age             exam.result     
 Min.   :1.00   Min.   : 1.000   vivax     :2365942  
 1st Qu.:1.00   1st Qu.: 3.000   falciparum: 262439  
 Median :1.00   Median : 4.000   negative  :      0  
 Mean   :1.39   Mean   : 4.423   F+FG      :      0  
 3rd Qu.:2.00   3rd Qu.: 6.000   F+V       :      0  
 Max.   :3.00   Max.   :10.000   V+FG      :      0  
                                 (Other)   :      0  

In [7]:
#agora só dados não numéricos
head(split)

,gender,age,exam.result
,<int>,<int>,<fct>
1,1,3,vivax
2,1,4,vivax
3,1,4,vivax
4,1,4,vivax
5,2,5,vivax
6,1,4,vivax


In [8]:
#reduzir a amostragem para 1000 registros

split <- sample_n(split, 1000)

In [9]:
# Aplicando o ALgoritmo Clustering do DBScan

# setup clustering
model <- cluster_dbscan(minPts = 10)
# build model
model <- fit(model, split[,1:2])
clu <- cluster(model, split[,1:2])
table(clu)




clu
  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15 
 24  12  71  44 121 128  77  53  42  69  67  98 123  19  22  30 

In [10]:
# evaluate model using external metric
eval <- evaluate(model, clu, split$exam.result)
eval

x,ce,qtd,ceg
<fct>,<dbl>,<int>,<dbl>
0,0.4138169,24,0.009931604
1,0.6500224,12,0.007800269
2,0.6220403,71,0.044164862
3,0.1564911,44,0.006885607
4,0.5855360,121,0.070849853
5,0.5212220,128,0.066716418
6,0.4812257,77,0.037054376
7,0.5631028,53,0.029844450
8,0.5266171,42,0.022117917


In [11]:
# Aplicando o ALgoritmo Clustering do k-means

modelk <- cluster_kmeans(k=3)

# build model
modelk <- fit(model, split[,1:2])
cluk <- cluster(model, split[,1:2])
table(cluk)
cluk

# evaluate model using external metric
evalk <- evaluate(modelk, cluk, split$exam.result)
evalk

cluk
  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15 
 24  12  71  44 121 128  77  53  42  69  67  98 123  19  22  30 

[1]  1  2  3  4  2  5  6  2  2  7  4  7  1  3  8  9  7 10 11  3  4 11 12  7
  [25]  4  6  0 13 14  5  9 12  9 12 14  6  8 12  4  4  6  7 10  4  2 12  6 13
  [49]  9 15  8 11  5  7  5  4 12  4  2  5  7  3 10 13  9 10  5  6  4 15 11  4
  [73]  2 12  8 10 10 10  5 12 12  6 10  3  7  0  5 13  9  3 12  6  8 12 11  6
  [97]  9  5 15  9  5 12  5  4  3  7 11  9 10  8 10 12  5  6  2  4 12  2  2 12
 [121] 11 12  0  5  2  5  1  5  3 12 11 14  4  5  9  5 12 11  2 10 10  4 11  3
 [145] 12 12 11  4 11  4 11  5  4  4  8  9  5  7 11 15  3  5  5 10  9 12  9  2
 [169] 11  7  8  2  8  0 12  5 12  2 12  4 11  6 12  7  9 10  5 12  5  5 11  5
 [193] 15  9 11  0 11  3  2  4  0  7  4  2  5  6 15  5  4 11 11  4  5  2  4  0
 [217]  9  4  8 11  4 12 10 13  5  2  4  4 10  3 10 11 15 15 12  4  2 11  8  9
 [241]  6 12 10  4  4 10  6  7  6  4 10  9  4  8  1 15  5  8 11  4  9  4  4  2
 [265] 11  6 10 11 11  6  7 11 10 11  7 12  5 10  5 12 10  9 10  3 11  5  1  7
 [289] 11  0  8 10  6  6 14  4  4 12  6 11 11  9  2  9  9 12 11  5  5 12  4  8
 [313]  5  6  4  4 10  4  4  0  1 10  7  2  4 10  7  5 11  2  5 10 12  5  7  2
 [337]  4 12  5  4  7 12 12  5  2  5  3 14  9  9 11  9  4  5  7 10 13  7  3 11
 [361]  8  4 11  4  4 11 15 12  2 12  4  5 12 10  5  4 15 11 10 12 14 11  2 11
 [385] 12  6  6 12 14  3  7 14 10  4 14  5 14  4  0 15 14  5  6  5  9  5  8  5
 [409] 10 10 11  5 11  1 12  5 10 12  4 12  3  5 11  5 12  9 12 11  4  8  9  9
 [433]  3  5  6  7  0  4 11  9  2  9 10  9  9  2 13  5 12 11 12  4  9  2 10  4
 [457] 14  6 11 11  4 12  1  0 15  5  4 10  5  2 12  7 12  4  4  9 14 10  5 12
 [481] 13  6  0  3 12  2  6  8  6  6  5 12 10  1 12 10 12  9  5 15  4 14  4 12
 [505]  9  9 12 12  2 10  4  6  5  8  9  3  5 15  4  4  3  9 10  7  7  5  6  4
 [529] 14  6  4 11 12 12 10  4  5  7 13  4  4 15 12  4  5  2  6  5 12  4  4 12
 [553]  8  5  8 13  2 11  5 12  5  6 12 12  0 11 12  4  4 12 11 10 12 10  4  6
 [577] 15  4  5  5  4  5  6 13 10  5  8 14  4 11  7  3  5  5 11  3  6 12  2  5
 [601]  5 10  2  6 15 11 11  5  5  5  6 11 12 12  8  0 12 12  9 10  7  4 12 12
 [625]  6  5  7  6 12 12 15  3 12 11  6  6 12 12 12  9  4  2 11 11  7 10  6  5
 [649]  6 11 14  7  5  5  6  4  3  5  2  5  6  4 11  5  4  7  7  4  7  3  6  2
 [673]  5 12  8  7  2 14  6 15  8  5  2 11  2 11 10  5 11  2  3 15  4 12  4 10
 [697]  4  5  8  4 14  8 12 10  4 12  2  5 15 12  2 11  4  9  9  9 11  6  4  2
 [721] 10 12  5  2  5  2 12 12  4  6  7  7 11  4  7 12 10  7  8  0  0 14  5  9
 [745] 11  5  9  6 14  2  3  3 13  5  4  4  4  6 11  2  5  3 12  6  6  0  3  9
 [769]  5 12 12  7 12  7 13  7  6 11  5 10  3 11  7  5  9  5  6  3  6  8  0  4
 [793] 10  3  6  4  5  4  9  7 13  5  8 15  6 12 11  6  0  0  2  6 15  3  7  2
 [817] 12  6  9  6 14  0  2 11  5 12  5  9  4  8 11 12  9 12  8  5  4 10  2  4
 [841]  9  2 11  5  2 10 12  2  4  7 11  2  4 12 12 11  2  8  4  9  5 12  0 15
 [865] 15  4 10  5  3 11  5  2 11 11 15 11 10 12  8  6  4  7 11  8 11  6  1  9
 [889]  1  9  5 10  9  6 13  3  7  3  5 10  4  4  2 11  7 12 11  3 11 11 10  4
 [913]  5  5 11  4 11 12  8  3  2 12 12 11  5  9 12 10 12 12 12  3 11  9  5  0
 [937]  5  9  8  9 12  2 12  2  9 12  6  6  4 11  8 13  6  2 11 15  8  3 13  5
 [961]  9  6  5  4  6  4 12 13 15  9  9  8  1  5  2  4  5  5  2  9  5  5 12  6
 [985]  6 12  3  2  4  6  7  3 15  9 11 13  8  2  5  6
attr(,"metric")
[1] 24

x,ce,qtd,ceg
<fct>,<dbl>,<int>,<dbl>
0,0.4138169,24,0.009931604
1,0.6500224,12,0.007800269
2,0.6220403,71,0.044164862
3,0.1564911,44,0.006885607
4,0.5855360,121,0.070849853
5,0.5212220,128,0.066716418
6,0.4812257,77,0.037054376
7,0.5631028,53,0.029844450
8,0.5266171,42,0.022117917


In [12]:
# Aplicando o ALgoritmo de tunning de hyperparametro no k-means

# fit model
modelt <- clu_tune(cluster_kmeans(k = 0))
ranges <- list(k = 1:10)
modelt <- fit(modelt, split[,1:2], ranges)
print ("Parâmetro escolhido: ", modelt$k)

# run model
clut <- cluster(modelt, split[,1:2])
table(clut)
# evaluate model using external metric
evalt <- evaluate(modelt, clut, split$exam.result)
evalt

[1] "Parâmetro escolhido: "


clut
  1   2   3   4 
383 125  73 419 

x,ce,qtd,ceg
<fct>,<dbl>,<int>,<dbl>
1,0.5589400,383,0.21407402
2,0.1633455,125,0.02041819
3,0.6446940,73,0.04706267
4,0.4920190,419,0.20615598


In [13]:
# Aplicando o ALgoritmo Clustering de PAM

# setup clustering
modelp <- cluster_pam(k=3)
# build model
modelp <- fit(modelp, split[,1:2])
clup <- cluster(modelp, split[,1:2])
table(clup)
clup

# evaluate model using external metric
evalp <- evaluate(modelp, clup, split$exam.result)
evalp


clup
  1   2   3 
148 506 346 

[1] 1 2 3 2 2 2 2 2 2 1 2 1 1 3 2 3 1 2 3 3 2 3 3 1 2 2 3 1 1 2 3 3 3 3 1 2 2
  [38] 3 2 2 2 1 2 2 2 3 2 1 3 1 2 3 2 1 2 2 3 2 2 2 1 3 2 1 3 2 2 2 2 1 3 2 2 3
  [75] 2 2 2 2 2 3 3 2 2 3 1 1 2 1 3 3 3 2 2 3 3 2 3 2 1 3 2 3 2 2 3 1 3 3 2 2 2
 [112] 3 2 2 2 2 3 2 2 3 3 3 3 2 2 2 1 2 3 3 3 1 2 2 3 2 3 3 2 2 2 2 3 3 3 3 3 2
 [149] 3 2 3 2 2 2 2 3 2 1 3 1 3 2 2 2 3 3 3 2 3 1 2 2 2 1 3 2 3 2 3 2 3 2 3 1 3
 [186] 2 2 3 2 2 3 2 1 3 3 3 3 3 2 2 3 1 2 2 2 2 1 2 2 3 3 2 2 2 2 3 3 2 2 3 2 3
 [223] 2 1 2 2 2 2 2 3 2 3 1 1 3 2 2 3 2 3 2 3 2 2 2 2 2 1 2 2 2 3 2 2 1 1 2 2 3
 [260] 2 3 2 2 2 3 2 2 3 3 2 1 3 2 3 1 3 2 2 2 3 2 3 2 3 3 2 1 1 3 3 2 2 2 2 1 2
 [297] 2 3 2 3 3 3 2 3 3 3 3 2 2 3 2 2 2 2 2 2 2 2 2 3 1 2 1 2 2 2 1 2 3 2 2 2 3
 [334] 2 1 2 2 3 2 2 1 3 3 2 2 2 3 1 3 3 3 3 2 2 1 2 1 1 3 3 2 2 3 2 2 3 1 3 2 3
 [371] 2 2 3 2 2 2 1 3 2 3 1 3 2 3 3 2 2 3 1 3 1 1 2 2 1 2 1 2 3 1 1 2 2 2 3 2 2
 [408] 2 2 2 3 2 3 1 3 2 2 3 2 3 3 2 3 2 3 3 3 3 2 2 3 3 3 2 2 1 1 2 3 3 2 3 2 3
 [445] 3 2 1 2 3 3 3 2 3 2 2 2 1 2 3 3 2 3 1 1 1 2 2 2 2 2 3 1 3 2 2 3 1 2 2 3 1
 [482] 2 1 3 3 2 2 2 2 2 2 3 2 1 3 2 3 3 2 1 2 1 2 3 3 3 3 3 2 2 2 2 2 2 3 3 2 1
 [519] 2 2 3 3 2 1 1 2 2 2 1 2 2 3 3 3 2 2 2 1 1 2 2 1 3 2 2 2 2 2 3 2 2 3 2 2 2
 [556] 1 2 3 2 3 2 2 3 3 1 3 3 2 2 3 3 2 3 2 2 2 1 2 2 2 2 2 2 1 2 2 2 1 2 3 1 3
 [593] 2 2 3 3 2 3 2 2 2 2 2 2 1 3 3 2 2 2 2 3 3 3 2 3 3 3 3 2 1 2 3 3 2 2 1 2 3
 [630] 3 1 3 3 3 2 2 3 3 3 3 2 2 3 3 1 2 2 2 2 3 1 1 2 2 2 2 3 2 2 2 2 2 3 2 2 1
 [667] 1 2 1 3 2 2 2 3 2 1 2 1 2 1 2 2 2 3 2 3 2 2 3 2 3 1 2 3 2 2 2 2 2 2 1 2 3
 [704] 2 2 3 2 2 1 3 2 3 2 3 3 3 3 2 2 2 2 3 2 2 2 2 3 3 2 2 1 1 3 2 1 3 2 1 2 1
 [741] 1 1 2 3 3 2 3 2 1 2 3 3 1 2 2 2 2 2 3 2 2 3 3 2 2 1 3 3 2 3 3 1 3 1 1 1 2
 [778] 3 2 2 3 3 1 2 3 2 2 3 2 2 3 2 2 3 2 2 2 2 3 1 1 2 2 1 2 3 3 2 3 1 2 2 1 3
 [815] 1 2 3 2 3 2 1 3 2 3 2 3 2 3 2 2 3 3 3 3 2 2 2 2 2 2 3 2 3 2 2 2 3 2 2 1 3
 [852] 2 2 3 3 3 2 2 2 3 2 3 1 1 1 2 2 2 3 3 2 2 3 3 1 3 2 3 2 2 2 1 3 2 3 2 1 3
 [889] 1 3 2 2 3 2 1 3 1 3 2 2 2 2 2 3 1 3 3 3 3 3 2 2 2 2 3 2 3 3 2 3 2 3 3 3 2
 [926] 3 3 2 3 3 3 3 3 3 2 1 2 3 2 3 3 2 3 2 3 3 2 2 2 3 2 1 2 2 3 1 2 3 1 2 3 2
 [963] 2 2 2 2 3 1 1 3 3 2 1 2 2 2 2 2 2 3 2 2 3 2 2 3 3 2 2 2 1 3 1 3 3 1 2 2 2
[1000] 2
attr(,"metric")
[1] 14513

x,ce,qtd,ceg
<fct>,<dbl>,<int>,<dbl>
1,0.6064482,148,0.08975433
2,0.5531954,506,0.27991687
3,0.3526253,346,0.12200836


In [44]:
#Tabela Comparativa dos Resultados de Cada Algoritmo

# recebe os resultados de cada algoritmo e no final agrupa os resultados em um DataFrame
result1 <- data.frame(X=c("DBScan"),Cluster_Entropy=eval$clustering_entropy,Data_Entropy=eval$data_entropy)
result2 <- data.frame(X=c("K-Means"),Cluster_Entropy=evalk$clustering_entropy,Data_Entropy=evalk$data_entropy)
result3 <- data.frame(X=c("Tunned K-Means"),Cluster_Entropy=evalt$clustering_entropy,Data_Entropy=evalt$data_entropy)
result4 <- data.frame(X=c("PAM"),Cluster_Entropy=evalp$clustering_entropy,Data_Entropy=evalp$data_entropy)
total <- bind_rows(result1, result2, result3, result4)

head(total)

,X,Cluster_Entropy,Data_Entropy
,<chr>,<dbl>,<dbl>
1,DBScan,0.4869145,0.499916
2,K-Means,0.4869145,0.499916
3,Tunned K-Means,0.4877109,0.499916
4,PAM,0.4916796,0.499916
